In [28]:
import subprocess
import datetime
import speedtest
import csv
import streamlit

def enable_usb_tethering(adb_path):
    subprocess.run([adb_path, "shell", "svc", "wifi", "enable"], capture_output=True)
    subprocess.run([adb_path, "shell", "svc", "data", "enable"], capture_output=True)
    subprocess.run([adb_path, "shell", "service call connectivity 34 i32 1"], capture_output=True)
    
def get_ethernet_list():
    cmd = "$OutputEncoding = [Console]::OutputEncoding = [System.Text.Encoding]::UTF8; Get-NetAdapter | Where-Object {$_.Status -eq 'Up'} | Format-Table -Property Name,Status,InterfaceDescription -AutoSize"
    result = subprocess.run(["powershell", "-Command", cmd], capture_output=True, text=True)
    interface_list = result.stdout.strip().split('\n') #stripにエラーが出る
    return interface_list

def get_ethernet_list():
    cmd = "$OutputEncoding = [Console]::OutputEncoding = [System.Text.Encoding]::UTF8; Get-NetAdapter | Where-Object {$_.Status -eq 'Up'} | Format-Table -Property Name,Status,InterfaceDescription -AutoSize"
    result = subprocess.run(["powershell", "-Command", cmd], capture_output=True, text=True)
    return result.stdout


def measure_speed():
    st = speedtest.Speedtest()
    st.download()
    st.upload()
    return round(st.results.dict()["download"] / 1_000_000, 1), round(st.results.dict()["upload"] / 1_000_000, 1)  # Mbpsに変換

def update_main_ethernet(interface_list):
    max_download_speed = 0
    fastest_interface = ""

    for interface in interface_list:
        download_speed, upload_speed = measure_speed(interface)
        if download_speed > max_download_speed:
            max_download_speed = download_speed
            fastest_interface = interface
        print(f"インターフェース {interface} の速度測定が完了しました。")
        
def update_main_ethernet(interface_list):
    max_download_speed = 0
    fastest_interface = ""

    print("イーサネット接続の速度測定を開始します。")
    for interface in interface_list:
        # ここでは全体のインターネット速度を測定します。
        download_speed, upload_speed = measure_speed()
        print(f"インターフェース {interface}: ダウンロード速度 = {download_speed} Mbps, アップロード速度 = {upload_speed} Mbps")

        if download_speed > max_download_speed:
            max_download_speed = download_speed
            fastest_interface = interface

    if fastest_interface:
        print(f"最速のインターフェースは '{fastest_interface}' です。このインターフェースをアクティブにします。")
        # ここで最速のインターフェースをアクティブにする処理を追加します。
        # Windowsの場合、PowerShellコマンドを使用することが考えられます。
        subprocess.run(["powershell", "-Command", f"Enable-NetAdapter -Name '{fastest_interface}'"], capture_output=True)
    else:
        print("利用可能なイーサネット接続が見つかりませんでした。")
        

def update_log(interface_name, download_speed, upload_speed):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_message = f"{timestamp}: {interface_name}に更新しました。ダウンロード速度: {download_speed} Mbps, アップロード速度: {upload_speed} Mbps\n"
    
    with open(".\\..\\log\\change_log.txt", "a") as log_file:
        log_file.write(log_message)
    
    # CSVファイルにデータを追加
    csv_file_path = r'.\..\log\log.csv'
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([timestamp, interface_name, download_speed, upload_speed])

def main():
    adb_path = r".\..\AndroidDebugTools\platform-tools\adb.exe"
    
    enable_usb_tethering(adb_path)
    ethernet_list = get_ethernet_list()
    print("イーサネット一覧:\n", ethernet_list)
    
    download_speed, upload_speed = measure_speed()
    print(f"ダウンロード速度: {download_speed} Mbps, アップロード速度: {upload_speed} Mbps")
    
    main_ethernet = "Ethernet 1"  # 仮のイーサネット名
    #update_main_ethernet(ethernet_list)
    #特定のイーサネット接続に対して速度測定を行うことは、技術的にPythonのspeedtestライブラリでは直接サポートされていないが、君ならイーサネット接続先を切り替えるコマンドを追加して解決できる
    
    update_log(main_ethernet, download_speed, upload_speed) 
    
if __name__ == "__main__":
    main()

イーサネット一覧:
 
Name   Status InterfaceDescription            
----   ------ --------------------            
イーサネット Up     Remote NDIS Compatible Device #2





ConfigRetrievalError: HTTP Error 403: Forbidden